In [ ]:
!pip install pycaret
!pip install joblib
!pip install pandas
!pip install sklearn

In [ ]:
#드라이브 미 사용시 코드 블록 제거
from google.colab import drive
drive.mount('/content/drive')

In [14]:
import joblib as jl
from pycaret.classification import *
import pandas as pd

In [36]:
model = jl.load('Q2_xgb.pkl')
t = pd.read_csv('Q2_test_최종.csv')
data = pd.read_csv('Q2_train_최종.csv')

In [37]:
clf = setup(data=data,
            target='root_cause_type',
            normalize = True,
            normalize_method = 'minmax',
            fold=4,
            fold_shuffle=True,
            session_id=23) #seed

,Description,Value
0,Session id,23
1,Target,root_cause_type
2,Target type,Multiclass
3,Original data shape,"(1114, 9)"
4,Transformed data shape,"(1114, 9)"
5,Transformed train set shape,"(779, 9)"
6,Transformed test set shape,"(335, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [38]:
pred = predict_model(model,t)
pred

,ticketno,alarmmsg_original_1,alarmmsg_original_2,alarmmsg_original_3,slotna,portna,time_len,msg_len,prediction_label,prediction_score
0,15238899.0,14,8,0,1,1,0.000000,4,2,0.9996
1,15712444.0,0,8,0,1,1,0.000000,2,2,0.9994
2,15723187.0,0,13,0,0,0,24.299999,3,2,0.9975
3,15737103.0,14,8,0,1,1,0.000000,4,2,0.9996
4,15737132.0,14,8,0,0,0,0.000000,4,2,0.9995
...,...,...,...,...,...,...,...,...,...,...
4322,22015278.0,4,0,0,0,0,0.000000,1,0,0.9985
4323,22015300.0,5,0,0,0,0,0.000000,1,0,0.9985
4324,23818326.0,79,0,0,1,1,0.150000,17,0,0.9908
4325,23819372.0,5,0,0,1,1,0.000000,1,0,0.9915


In [39]:
t['root_cause_type'] = pred['prediction_label']
t['root_cause_type'] = t['root_cause_type'].apply(lambda x: 'LinkCut' if x == 0 else ('UnitFail' if x == 1 else ('PowerFail' if x == 2 else x)))
result = t[['ticketno','root_cause_type']]

In [42]:
def submitResult(pred):
    try:
        label = pd.read_csv('Q2_label_sample.csv')
        # ticketno 순서와 개수가 일치하는지 확인
        if (label['ticketno'] == result['ticketno']).all():
            print("Check: ticketno 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 ticketno가 일치하지 않습니다.")
            return

        pred.to_csv('Q2_submitResult.csv', index=False)
        print("Done : Q2_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

submitResult(result)

Check: ticketno 순서와 샘플 수가 일치합니다.
Done : Q2_submitResult.csv 파일로 저장되었습니다.
